# Data Cleaning and Visualization

There are 23 columns and 145460 rows in this time-series dataset. The goal is to predict the `RainTomorrow` variable.

In [63]:
import pandas as pd
import numpy as np

# helper function to get quick row info
def getDfRowInfo(dataframe):
    print('DF Row Info:')
    print('\tTotal number of rows:', len(df.index))
    print('\tPercentage of original number of rows:', len(df.index) / 145460)
    print('\tTotal number of rows with the RainTomorrow = true:', len([x for x in df['RainTomorrow'] if x == 1]))
    print('\tPercentage of dataset that has RainTomorrow = true:', len([x for x in df['RainTomorrow'] if x == 1]) / len(df.index))

# -----------------------
# read in dataset with date column parsed
df = pd.read_csv('weatherAUS.csv',
    parse_dates=['Date'],
    index_col='Date')

# transform binary string to binary 0 or 1 values
df['RainToday'] = df['RainToday'].transform(lambda x: 0 if x == 'No' else 1)
df['RainTomorrow'] = df['RainTomorrow'].transform(lambda x: 0 if x == 'No' else 1)

getDfRowInfo(df)


# ------------------------
# print out the number of non-NA values per column
print('Total number of non-NA values per column:')
print(df.count(0))

# TODO: what to do with rows that are NA values? remove columns that are half full of NAs? replace NAs with 0 or some value?
#       have two datasets to compare?

# remove columns with mostly NA values?: Sunshine, Evaporation, Cloud9am, Cloud3pm
df = df.drop(['Sunshine', 'Evaporation', 'Cloud9am', 'Cloud3pm'], axis=1)
getDfRowInfo(df)

 
# remove rows that have any NA values? - no, this removes 61% of the rows before removing the 4 columns above
#       after removing the above 4 columns, it removes 22% of the rows
df = df.dropna()
getDfRowInfo(df)

# -------------------------
# TODO: what to do with location column that is string geographic data? does it matter, should we drop it? 
#       should we split dataset by location?
# TODO: what to do with wind direction columns that are 17 enumerations of strings of cardinal directions, including NaN?
print('Unique wind direction values:', df['WindGustDir'].unique())

# -------------------------
# preview data
# df.head()
# print(df.describe())

DF Row Info:
	Total number of rows: 145460
	Percentage of original number of rows: 1.0
	Total number of rows with the RainTomorrow = true: 35144
	Percentage of dataset that has RainTomorrow = true: 0.24160593977725836
Total number of non-NA values per column:
Location         145460
MinTemp          143975
MaxTemp          144199
Rainfall         142199
Evaporation       82670
Sunshine          75625
WindGustDir      135134
WindGustSpeed    135197
WindDir9am       134894
WindDir3pm       141232
WindSpeed9am     143693
WindSpeed3pm     142398
Humidity9am      142806
Humidity3pm      140953
Pressure9am      130395
Pressure3pm      130432
Cloud9am          89572
Cloud3pm          86102
Temp9am          143693
Temp3pm          141851
RainToday        145460
RainTomorrow     145460
dtype: int64
DF Row Info:
	Total number of rows: 145460
	Percentage of original number of rows: 1.0
	Total number of rows with the RainTomorrow = true: 35144
	Percentage of dataset that has RainTomorrow = true: 0

In [64]:
# write cleaned dataset to new file

df.to_csv('cleanedWeatherAUS.csv')

In [65]:
# do some initial analysis? data visualizations on the dataset content
# useful syntax to slice data based on time:
df["1/1/2010":"1/2/2010"].head() # gets all data in the given date range

,Location,MinTemp,MaxTemp,Rainfall,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
Date,,,,,,,,,,,,,,,,,,
2010-01-01,Albury,19.4,31.9,5.0,NNE,39.0,NW,WNW,9.0,9.0,70.0,40.0,1012.2,1008.5,23.4,30.9,1,1
2010-01-02,Albury,18.6,29.1,12.4,W,56.0,S,W,6.0,28.0,88.0,48.0,1007.8,1006.2,20.6,28.0,1,0
2010-01-01,BadgerysCreek,19.6,29.1,0.0,NNW,43.0,NNE,N,11.0,20.0,83.0,56.0,1014.2,1010.6,22.4,28.2,0,0
2010-01-02,BadgerysCreek,20.3,30.3,0.0,SE,41.0,NNE,NE,9.0,9.0,72.0,64.0,1007.9,1005.5,24.5,28.6,0,1
2010-01-01,Cobar,19.3,26.4,1.6,NE,30.0,NNE,NNE,13.0,19.0,96.0,86.0,1011.1,1008.9,21.3,23.8,1,1
